In [ ]:
pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import when
import pyspark.sql.functions as sf
import os
from pyspark.sql.functions import rank
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
from pyspark.sql.functions import collect_list

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
path = '/content/drive/MyDrive/log_search/'

In [ ]:
df = spark.read.parquet('/content/drive/MyDrive/log_search/20220601')

In [ ]:
df = df.select('datetime','user_id','keyword')

In [ ]:
df.show()

+--------------------+--------+--------------------+
|            datetime| user_id|             keyword|
+--------------------+--------+--------------------+
|2022-06-01 18:59:...|    null|            trữ tình|
|2022-06-01 18:59:...|44887906|            trữ tình|
|2022-06-01 18:59:...| 2719170|              bolero|
|2022-06-01 15:00:...|    null|amy schumer: trực...|
|2022-06-01 19:00:...| 8830996|cậu mang à sĩ hanako|
|2022-06-01 19:00:...|    null|      Hoa trong bao |
|2022-06-01 19:00:...|41559909|liên minh công lý...|
|2022-06-01 19:00:...|92715770|                null|
|2022-06-01 19:00:...|49026196|    việt nam vs appa|
|2022-06-01 19:00:...|    null|chuyển sinh thành...|
|2022-06-01 19:00:...|41376437|nhất kiến khuynh tâm|
|2022-06-01 19:00:...| 1254139|                giác|
|2022-06-01 19:00:...|42534799|            nexsport|
|2022-06-01 19:00:...|49190631|Tìm kiếm bằng giọ...|
|2022-06-01 19:00:...|    null|Lương Sơn Bá Chúc...|
|2022-06-01 19:00:...|91485275|một mảnh băng t

In [ ]:
data = os.listdir(path)
data.sort()

In [ ]:
list_t6 = data[0:14]

In [ ]:
list_t7 = data[14:]

In [ ]:
from pyspark.sql.types import StructType, StructField, TimestampType, StringType
schema = StructType([
    StructField("datetime", TimestampType(), True),
    StructField("user_id", StringType(), True),
    StructField("keyword", StringType(), True)
])

In [ ]:
data_t6 = spark.createDataFrame([], schema)

In [ ]:
for i in list_t6:
  df = spark.read.parquet(path+i)
  df = df.select('datetime','user_id','keyword')
  data_t6 = data_t6.union(df)
  print('Read data {}'.format(i))

Read data 20220601
Read data 20220602
Read data 20220603
Read data 20220604
Read data 20220605
Read data 20220606
Read data 20220607
Read data 20220608
Read data 20220609
Read data 20220610
Read data 20220611
Read data 20220612
Read data 20220613
Read data 20220614


In [ ]:
data_t6.count()

1146996

In [ ]:
data_t6.show()

+--------------------+--------+--------------------+
|            datetime| user_id|             keyword|
+--------------------+--------+--------------------+
|2022-06-01 18:59:...|    null|            trữ tình|
|2022-06-01 18:59:...|44887906|            trữ tình|
|2022-06-01 18:59:...| 2719170|              bolero|
|2022-06-01 15:00:...|    null|amy schumer: trực...|
|2022-06-01 19:00:...| 8830996|cậu mang à sĩ hanako|
|2022-06-01 19:00:...|    null|      Hoa trong bao |
|2022-06-01 19:00:...|41559909|liên minh công lý...|
|2022-06-01 19:00:...|92715770|                null|
|2022-06-01 19:00:...|49026196|    việt nam vs appa|
|2022-06-01 19:00:...|    null|chuyển sinh thành...|
|2022-06-01 19:00:...|41376437|nhất kiến khuynh tâm|
|2022-06-01 19:00:...| 1254139|                giác|
|2022-06-01 19:00:...|42534799|            nexsport|
|2022-06-01 19:00:...|49190631|Tìm kiếm bằng giọ...|
|2022-06-01 19:00:...|    null|Lương Sơn Bá Chúc...|
|2022-06-01 19:00:...|91485275|một mảnh băng t

In [ ]:
data_t6 = data_t6.filter(data_t6.user_id.isNotNull())
data_t6 = data_t6.filter(data_t6.keyword.isNotNull())

In [ ]:
data_t6.count()

790587

In [ ]:
data_t7 = spark.createDataFrame([], schema)
for i in list_t7:
  df = spark.read.parquet(path+i)
  df = df.select('datetime','user_id','keyword')
  data_t7 = data_t7.union(df)
  print('Read data {}'.format(i))

Read data 20220701
Read data 20220702
Read data 20220703
Read data 20220704
Read data 20220705
Read data 20220706
Read data 20220707
Read data 20220708
Read data 20220709
Read data 20220710
Read data 20220711
Read data 20220712
Read data 20220713
Read data 20220714


In [ ]:
data_t7.count()

1219976

In [ ]:
data_t7 = data_t7.filter(data_t7.user_id.isNotNull())
data_t7 = data_t7.filter(data_t7.keyword.isNotNull())
data_t7.count()

855321

In [ ]:
most_search_t6 = data_t6.select('user_id','keyword')
most_search_t6 = most_search_t6.groupby('user_id','keyword').count()
most_search_t6 = most_search_t6.withColumnRenamed('count','TotalSearch')

In [ ]:
most_search_t6.orderBy('user_id', ascending=False).show()

+--------+--------------------+-----------+
| user_id|             keyword|TotalSearch|
+--------+--------------------+-----------+
|99999941|       cẩm tú nam ca|          1|
|99999941|             vi uong|          1|
|99999941|    thien menh hoang|          1|
|99999941|          thien mẻnh|          1|
|99999941|lấy danh nghĩa ng...|          1|
|99999941|        linh hon bac|          1|
|99999941|             gintama|          1|
|99999793|      sở kiều truyện|          1|
|99999793|cô nàng trong trắ...|          1|
|99999793|               conan|          2|
|99999793|     bằng chứng thép|          1|
|99999793|       thám tử conan|          1|
|99999793|     vân tịch truyện|          4|
|99999793|    như ý phương phi|          1|
|99999793| hải thượng phồn hoa|          1|
|99999793|cuối cùng cũng đế...|          1|
|99999759|bí mật không thể nói|          1|
|99999759|cô nàng trong trắ...|          3|
|99999351|     phim thanh xuân|          1|
|99999351|thanh xuân năm ấy...| 

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
windowSpec = Window.partitionBy('user_id').orderBy(col('TotalSearch').desc())

In [ ]:
most_search_t6 = most_search_t6.withColumn("Row_Number", row_number().over(windowSpec))
most_search_t6 = most_search_t6.filter(most_search_t6.Row_Number==1)

In [ ]:
most_search_t6 = most_search_t6.select('user_id','keyword')
most_search_t6 = most_search_t6.withColumnRenamed('keyword','most_search_t6')
most_search_t6.show()

+-------+--------------------+
|user_id|      most_search_t6|
+-------+--------------------+
|0000402|tieu nuong tu nha...|
|0000825|    cùng nhau ăn lẩu|
|0000924|      nơi vì sao rơi|
| 000104|                 HAU|
|0001969|attack on titan (...|
|0002189|toà án vị thành niên|
|0002235|nhanh và nguy hiểm 9|
|0003361|         tây hành kỷ|
|0003371|                 vtv|
|0003691| thế giới khủng long|
|0004007| tom and jerry tales|
|0004287|          thuyếtminh|
|0005178|              BORUTO|
|0005186|hạnh phúc trong t...|
|0005187|          Kim Tae Ri|
|0005748|   lời nguyền ma lai|
|0006530|us women's open 2...|
|0006594|   phim mộng hoa lục|
|0006773|                  30|
|0007284| hen ho chon cong so|
+-------+--------------------+
only showing top 20 rows



In [ ]:
most_search_t7 = data_t7.select('user_id','keyword')
most_search_t7 = most_search_t7.groupby('user_id','keyword').count()
most_search_t7 = most_search_t7.withColumnRenamed('count','TotalSearch')
most_search_t7 = most_search_t7.withColumn("Row_Number", row_number().over(windowSpec))
most_search_t7 = most_search_t7.filter(most_search_t7.Row_Number==1)
most_search_t7 = most_search_t7.select('user_id','keyword')
most_search_t7 = most_search_t7.withColumnRenamed('keyword','most_search_t7')
most_search_t7.show()

+-------+--------------------+
|user_id|      most_search_t7|
+-------+--------------------+
|0000699|             thả thí|
|0000786|theo dấu vương qu...|
|0001109|hương mật tựa khó...|
|0001213|Liên Minh Công Lý...|
|0001465|  viet nam vs mianma|
|0001487|      con tim sắt đá|
|0001606|hương mật tựa khó...|
|0001969|               GIO P|
|0003361|         lộc đỉnh ký|
|0003371|hoang anh gia lai...|
|0003437|phương trình tình...|
|0003691|tiếng gọi nơi hoa...|
|0004007|            doraemon|
|0004285|                 eve|
|0004521|          CHEN THANH|
|0004675|        container 39|
|0004999|   vụ mất tích bí ẩn|
|0005187|high school king ...|
|0005566|        U19 THAI LAN|
|0005748|chuyen sinh thanh...|
+-------+--------------------+
only showing top 20 rows



In [ ]:
result = most_search_t6.join(most_search_t7,'user_id','inner')

In [ ]:
result.repartition(1).write.csv('/content/drive/MyDrive/log_search', header=True,mode="overwrite")

##Summary Trending

In [ ]:
trend_t6 = data_t6.select('datetime','keyword')

In [ ]:
from pyspark.sql.functions import col, date_format
from pyspark.sql.types import DateType
trend_t6 = trend_t6.withColumn("datetime", date_format(col("datetime"), "yyyy-MM-dd").cast(DateType()))

In [ ]:
trend_t6 = trend_t6.groupby('keyword').count().orderBy('count',ascending=False)

In [ ]:
trend_t7 = data_t7.select('datetime','keyword')
trend_t7 = trend_t7.withColumn("datetime", date_format(col("datetime"), "yyyy-MM-dd").cast(DateType()))
trend_t7 = trend_t6.groupby('keyword').count().orderBy('count',ascending=False)

In [ ]:
trend_t7.show()

+--------------------+-----+
|             keyword|count|
+--------------------+-----+
|    bong da viet nam|    1|
|                 vt |    1|
| Giao hữu bóng đá VN|    1|
|   gái già lắm chiêu|    1|
|      phim lưới trời|    1|
|    VIET NAM VS AFGH|    1|
|   nữ vệ sĩ sinh dep|    1|
|     bóng dá hôm nay|    1|
|biệt đội thiếu ni...|    1|
|guardians of the ...|    1|
|     bóng đá tối nay|    1|
|cuộc chiến thượng...|    1|
|Cô Nàng Đầu Bếp C...|    1|
|                GAME|    1|
| da bong truc tie...|    1|
|    mị nguyệt truyện|    1|
|viet nam và afgan...|    1|
|    Phụng Hoàng lửa |    1|
|                   K|    1|
|trực tiếp bóng đá...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
trend_t6.show()

+--------------------+-----+
|             keyword|count|
+--------------------+-----+
|Liên Minh Công Lý...| 8741|
|          fairy tail| 6771|
|  thiên nga bóng đêm| 5471|
|            sao băng| 4909|
| nữ thanh tra tài ba| 4895|
|       bắt ma phá án| 4837|
|           siêu nhân| 3938|
|        mộng hoa lục| 3411|
|cuộc chiến thượng...| 3001|
|cô nàng trong trắ...| 2950|
|vô tình nhặt được...| 2646|
|         hội pháp sư| 2556|
|tôi thấy hoa vàng...| 2540|
|            why her?| 2449|
|lấy danh nghĩa ng...| 2443|
|bạn trai tôi là h...| 2304|
|cảnh đẹp ngày vui...| 2262|
|           chàng hậu| 2226|
|    yêu nhầm chị dâu| 2179|
|Tìm kiếm bằng giọ...| 2105|
+--------------------+-----+
only showing top 20 rows

